In [ ]:
from poppy.creatures import PoppyTorso
import numpy as np
import time
import math

poppy = PoppyTorso(simulator='vrep')

# Add object
io = poppy._controllers[0].io
name = 'cube'
position = [0, -0.15, 0.85] # X, Y, Z
sizes = [0.1, 0.1, 0.1] # in meters
mass = 0 # in kg
io.add_cube(name, position, sizes, mass)
time.sleep(1)
name1 = 'cube2'
position1 = [0, -1, 0.5]
sizes1 = [3, 0.3, 1]
io.add_cube(name1, position1, sizes1, mass)
io.set_object_position('cube', position=[0, -1, 1.05])

In [ ]:

############################ Pseudo CV algorithm ##############################

# Output the heading direction of camera

def headForwardDirection():
    angleNegativeY = poppy.head_z.present_position
    angleSurfaceXY = - poppy.head_y.present_position

    angleNegativeY = angleNegativeY / 180 * 3.14159
    angleSurfaceXY = angleSurfaceXY / 180 * 3.14159

    y = - np.cos(angleSurfaceXY) * np.cos(angleNegativeY)
    x = np.cos(angleSurfaceXY) * np.sin(angleNegativeY)
    z = np.sin(angleSurfaceXY)

    forwardDire = [x, y, z]
    return forwardDire

# Output the relative position of object w.r.t camera position

def objectRelPosition(name):
    objectPos = io.get_object_position(name)
    positionCameraOri = [0, -0.05, 1.06] # Camera's position in Vrep

    objectRelPos = [objectPos[i] - positionCameraOri[i] for i in xrange(3)]

    return objectRelPos

# Output the angels(horizontal, vertical) of object w.r.t the center of camera sight
# If object is not in sight, return False

def canSeeJudge(headForwardDirection, objectRelPos):
    orthognalBasis1 = headForwardDirection
    orthognalBasis2 = [orthognalBasis1[1], -orthognalBasis1[0], 0]
    normOrthBasis2 = np.linalg.norm(orthognalBasis2)
    orthognalBasis2 =  [orthognalBasis2[i] / normOrthBasis2 for i in xrange(3) ]
    orthognalBasis3 = np.cross(orthognalBasis2, orthognalBasis1)

    objectProjectionOnOrthBasis1 = np.dot(objectRelPos, orthognalBasis1)
    if objectProjectionOnOrthBasis1 < 0:
        return False
    objectProjectionOnOrthBasis2 = np.dot(objectRelPos, orthognalBasis2)
    objectProjectionOnOrthBasis3 = np.dot(objectRelPos, orthognalBasis3)

    newCoordinate = [objectProjectionOnOrthBasis1, objectProjectionOnOrthBasis2, objectProjectionOnOrthBasis3]

    tt = [1, 0, 0]

    t = [objectProjectionOnOrthBasis1, objectProjectionOnOrthBasis2, 0]
    angle1 = np.arccos(np.dot(tt, t) / np.linalg.norm(t)) / 3.14159 * 180

    if abs(angle1) > 37:
        return False
    t = [objectProjectionOnOrthBasis1, 0, objectProjectionOnOrthBasis3]
    angle2 = np.arccos(np.dot(tt, t) / np.linalg.norm(t)) / 3.14159 * 180
    if abs(angle2) > 18.5:
        return False
    
    if objectProjectionOnOrthBasis3 < 0 and angle2 > 0:
        angle2 = -angle2

    if objectProjectionOnOrthBasis2 > 0 and angle1 > 0:
        angle1 = -angle1
    return angle1, angle2

# Output the state of problem
def getState(angle, m, n):
    if not angle:
        print "The object can't be seen"
        return False
    angle1, angle2 = angle
    state1 = math.ceil(abs(np.sin(angle1 / 180.0 * 3.14159) / np.sin(37 / 180.0 * 3.14159) * m))
    state2 = math.ceil(abs(np.sin(angle2 / 180.0 * 3.14159) / np.sin(18.5 / 180.0 * 3.14159) * m))
    if angle1 > 0:
        state1 = -state1
    if angle2 < 0:
        state2 = -state2
    return (state1, state2)

##############################################################################

In [ ]:
# Motor Action Configuration
def headMotionUp():
    angleY = poppy.head_y.present_position
    poppy.head_y.goal_position = angleY - 3
    time.sleep(0.02)
def headMotionDown():
    angleY = poppy.head_y.present_position
    poppy.head_y.goal_position = angleY + 3
    time.sleep(0.02)
def headMotionLeft():
    angleZ = poppy.head_z.present_position
    poppy.head_z.goal_position = angleZ + 3
    time.sleep(0.02)
def headMotionRight():
    angleZ = poppy.head_z.present_position
    poppy.head_z.goal_position = angleZ - 3
    time.sleep(0.02)
def headMotionUpLeft():
    angleY = poppy.head_y.present_position
    angleZ = poppy.head_z.present_position
    poppy.head_z.goal_position = angleZ + 3
    poppy.head_y.goal_position = angleY - 3
    time.sleep(0.02)
def headMotionUpRight():
    angleY = poppy.head_y.present_position
    angleZ = poppy.head_z.present_position
    poppy.head_z.goal_position = angleZ - 3
    poppy.head_y.goal_position = angleY - 3
    time.sleep(0.02)
def headMotionDownLeft():
    angleY = poppy.head_y.present_position
    angleZ = poppy.head_z.present_position
    poppy.head_z.goal_position = angleZ + 3
    poppy.head_y.goal_position = angleY + 3
    time.sleep(0.02)
def headMotionDownRight():
    angleY = poppy.head_y.present_position
    angleZ = poppy.head_z.present_position
    poppy.head_z.goal_position = angleZ - 3
    poppy.head_y.goal_position = angleY + 3
    time.sleep(0.02)